In [7]:
import torch
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import collections
import numpy as np
import pandas as pd
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import scipy
from scipy.io import loadmat as lm

from torch_geometric.utils.convert import  from_scipy_sparse_matrix

**ПРОБЛЕМЫ ВОПРОСЫ**:
- BlogCatalog, CoCit, Wiki имебют матрицу groupб утвреждается что это лейблы но как быть если у BlogCatalog, Wiki на каждую вершину по более чем одному лейблу -  поставила всем как - фичи
- Epinion, Arxiv не имеют ни лейблов ни фичей

In [8]:
def to_networkx(data, node_attrs=None, edge_attrs=None):
    r"""Converts a data object graph to a networkx graph.

    Args:
        data (torch_geometric.data.Data): The data object.
        node_attrs (iterable of str, optional): The node attributes to be
            copied. (default: :obj:`None`)
        edge_attrs (iterable of str, optional): The edge attributes to be
            copied. (default: :obj:`None`)
    """

    G = nx.Graph()
    if node_attrs is not None:
        
        for i,attr in enumerate(node_attrs):
            G.add_node(i, feature = str(attr.tolist()))
    else:
        G.add_nodes_from(list(range((data.num_nodes))))
    
    for i, (u, v) in enumerate(data.edge_index.t().tolist()):
        G.add_edge(u, v)
    
    return G


In [9]:
Blogcatalog = lm('datasets/blogcatalog.mat')
blogcatalog_edgeindex,_=from_scipy_sparse_matrix(Blogcatalog['network'])
blogcatalog_x = torch.tensor(scipy.sparse.csr_matrix.toarray(Blogcatalog['group']))
data_BlogCatalog = Data(edge_index = blogcatalog_edgeindex, x = blogcatalog_x)
G_BlogCatalog = to_networkx(data_BlogCatalog, blogcatalog_x)
#nx.from_scipy_\sparse_matrix(data_Blogcatalog['network'])

In [10]:
CoCit = lm('datasets/CoCit.mat')
CoCit_edgeindex,_=from_scipy_sparse_matrix(CoCit['network'])
CoCit_x = torch.tensor(scipy.sparse.csr_matrix.toarray(CoCit['group']))
data_CoCit = Data(edge_index = CoCit_edgeindex, x = CoCit_x)
G_CoCit=to_networkx(data_CoCit,)

In [11]:
Wiki = lm('datasets/Wiki.mat')
Wiki_edgeindex,_=from_scipy_sparse_matrix(Wiki['network'])
Wiki_x = torch.tensor(scipy.sparse.csr_matrix.toarray(Wiki['group']))
data_Wiki = Data(edge_index = Wiki_edgeindex, x = Wiki_x)
G_Wiki=to_networkx(data_Wiki,Wiki_x)

In [13]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(Blogcatalog['network']).transpose() == scipy.sparse.csr_matrix.toarray(Blogcatalog['network']))

True

In [14]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(CoCit['network']).transpose() == scipy.sparse.csr_matrix.toarray(CoCit['network']))

True

In [15]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(Wiki['network']).transpose() == scipy.sparse.csr_matrix.toarray(Wiki['network']))

True

In [16]:
with open('datasets/soc-Epinions1.txt','r') as f:
    data_Epinion = f.readlines()

new_list=list(map(lambda x: x.split('\t'),data_Epinion[4:]))
#print(new_list)
dataedge_ep = list(map(lambda x: [int(x[0]),int(x[1].split('\n')[0])],new_list))
set1=set(collections.Counter((np.array(dataedge_ep)).transpose()[1]).keys()) | set(collections.Counter((np.array(dataedge_ep)).transpose()[0]).keys())
map_ep={}
for j,i in enumerate(set1):
    map_ep[i] = j
dataedge_ep_new = list(map(lambda x: [map_ep[x[0]],map_ep[x[1]]], dataedge_ep))
data_edge_index_Epinion = torch.tensor(dataedge_ep_new).t()
data_Eppinion_torch = Data(edge_index=data_edge_index_Epinion)
G_Epinion = to_networkx(data_Eppinion_torch)

In [17]:
with open('datasets/CA-GrQc.txt','r') as f:
    data_Arxiv = f.readlines()

new_list=list(map(lambda x: x.split('\t'),data_Arxiv[4:]))
#print(new_list)
dataedge = list(map(lambda x: [int(x[0]),int(x[1].split('\n')[0])],new_list))
set2=set(collections.Counter((np.array(dataedge)).transpose()[1]).keys()) | set(collections.Counter((np.array(dataedge)).transpose()[0]).keys())
map_ar={}
for j,i in enumerate(set2):
    map_ar[i] = j
    
dataedge_ar_new = list(map(lambda x: [map_ar[x[0]],map_ar[x[1]]], dataedge))

data_edge_index_Arxiv = torch.tensor(dataedge_ar_new).t()
data_Arxiv_torch = Data(edge_index=data_edge_index_Arxiv)
G_Arxiv = to_networkx(data_Arxiv_torch)

In [18]:
from torch_geometric.datasets import Planetoid

data_Cora =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())[0]
#data_cora =dataset[0]
data_Citeseer =Planetoid(root='/tmp/Citeseer', name='Citeseer',transform=T.NormalizeFeatures())[0]
data_Pubmed =Planetoid(root='/tmp/Pubmed', name='Pubmed',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Amazon

data_Photo =Amazon(root='/tmp/Photo', name='Photo',transform=T.NormalizeFeatures())[0]
data_Computers =Amazon(root='/tmp/Computers', name='Computers',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Coauthor

data_CS =Coauthor(root='/tmp/CS', name='CS',transform=T.NormalizeFeatures())[0]
data_Physics =Coauthor(root='/tmp/Physics', name='Physics',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Reddit


#social networks: Reddit, Flickr
from torch_geometric.datasets import Flickr
data_Flickr =Flickr(root='/tmp/Flickr',transform=T.NormalizeFeatures())[0]
# data_Reddit = Reddit(root='/tmp/Reddit',transform=T.NormalizeFeatures()) очень тяжелый 

#Wikipedia "Chameleon", "Squirrel"

#Datasets of varying the homofily
from torch_geometric.datasets import MixHopSyntheticDataset
data_Homofily_0 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.0, transform=T.NormalizeFeatures())[0]
data_Homofily_01 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.1, transform=T.NormalizeFeatures())[0]
data_Homofily_02 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.2, transform=T.NormalizeFeatures())[0]
data_Homofily_03 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.3, transform=T.NormalizeFeatures())[0]
data_Homofily_04 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.4, transform=T.NormalizeFeatures())[0]
data_Homofily_05 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.5, transform=T.NormalizeFeatures())[0]
data_Homofily_06 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.6, transform=T.NormalizeFeatures())[0]
data_Homofily_07 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.7, transform=T.NormalizeFeatures())[0]
data_Homofily_08 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.8, transform=T.NormalizeFeatures())[0]
data_Homofily_09 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.9, transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import PPI

data_PPI = PPI(root='/tmp/PPI', split="train",transform=T.NormalizeFeatures())[0] #тут только train

In [19]:
datasets = [data_Cora, data_Citeseer, data_Pubmed, data_Photo, data_Computers, data_CS, data_Physics,data_Flickr,data_Homofily_0, data_Homofily_01,data_Homofily_02,data_Homofily_03,data_Homofily_04,data_Homofily_05,data_Homofily_06,data_Homofily_07,data_Homofily_08,data_Homofily_09,data_CoCit,data_BlogCatalog,data_Wiki]
datasets_names = ['Cora', 'Citeseer', 'Pubmed', 'Photo', 'Computers', 'CS', 'Physics','Flickr','Homophily0','Homophily01','Homophily02','Homophily03','Homophily04','Homophily05','Homophily06','Homophily07','Homophily08','Homophily09','CoCit','BlogCatalog','Wikipedia','Arxiv','Epinion']

In [20]:
graphs = []
for dataset in datasets:
    graphs.append(to_networkx(dataset,dataset.x))
graphs.append(G_Arxiv)
graphs.append(G_Epinion)


In [21]:
number_of_nodes = []
clustering_coef=[]
density=[]
degree_assort=[]
number_of_features=[]
number_of_labels = []
attribute_assort = []
isdirected=[]
 
for i,gr in enumerate(graphs):
        print(i)
        try:
            n_f=datasets[i].num_features
            number_of_features.append(n_f)
        except:
            if (datasets_names[i] == 'CoCit') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                number_of_features.append(datasets[i].shape(1))
            else:
                number_of_features.append(0)
        
        try:
            i_d=(datasets[i].is_directed())
            isdirected.append(i_d)
        except:
            if (datasets_names[i] == 'CoCit') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                isdirected.append(False)
            elif (datasets_names[i] == 'Arxiv') or (datasets_names[i]=='Epinion'):
                isdirected.append(True)
        
        try:
            n_l = len(collections.Counter(datasets[i].y.tolist()).keys())
            number_of_labels.append(n_l)
        except:
            if (datasets_names[i] == 'CoCit') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                number_of_labels.append(datasets[i].x.shape[1])
            elif (datasets_names[i] == 'Arxiv') or (datasets_names[i]=='Epinion'):
                number_of_labels.append(None)
        
        try:
            a_s = nx.attribute_assortativity_coefficient(gr,'feature')
            attribute_assort.append(a_s)
        except:
            attribute_assort.append(None)
            
        number_of_nodes.append(gr.number_of_nodes())
            #плотность
        density.append(nx.density(gr))
             #ассортативность
        degree_assort.append(nx.degree_assortativity_coefficient(gr))
        clustering_coef.append(np.mean(list(nx.clustering(gr).values())))
       

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\networkx\algorithms\assortativity\correlation.py:263: RuntimeWarning: invalid value encountered in double_scalars
  r = (t - s) / (1 - s)


In [22]:
df = pd.DataFrame({'Name of dataset' : datasets_names,
                                'Number of nodes (NN)' :number_of_nodes,'Number of features (NF)' :number_of_features,'Number of lables (NL)':number_of_labels , 'Directed? ' :isdirected,
                                'Clustering coefficient (CC)': clustering_coef,'Density(D)':density, 'Degree assortativity coefficient(DAC)':degree_assort, 'Attribute assortativity coefficient(AAC)': attribute_assort })
df.sort_values(by=['Attribute assortativity coefficient(AAC)']) #добавить атрибуты, направленность

,Name of dataset,Number of nodes (NN),Number of features (NF),Number of lables (NL),Directed?,Clustering coefficient (CC),Density(D),Degree assortativity coefficient(DAC),Attribute assortativity coefficient(AAC)
20,Wikipedia,4777,40,40.0,False,0.538570,0.008110,-0.237261,-0.015479
0,Cora,2708,1433,7.0,False,0.240673,0.001440,-0.065871,-0.000850
1,Citeseer,3327,3703,6.0,False,0.141471,0.000823,0.048378,-0.000547
3,Photo,7650,745,8.0,False,0.403979,0.004070,-0.044943,-0.000427
8,Homophily0,5000,2,10.0,False,0.009002,0.002384,-0.054907,-0.000410
9,Homophily01,5000,2,10.0,False,0.011408,0.002384,-0.053471,-0.000410
11,Homophily03,5000,2,10.0,False,0.012979,0.002384,-0.049402,-0.000405
10,Homophily02,5000,2,10.0,False,0.011378,0.002384,-0.051903,-0.000403
12,Homophily04,5000,2,10.0,False,0.014314,0.002384,-0.046596,-0.000402
13,Homophily05,5000,2,10.0,False,0.018294,0.002384,-0.049194,-0.000399


In [31]:
import pickle 
with open('analytics.pickle','wb') as f:
    pickle.dump(df,f)

In [ ]:

d_check = 

In [ ]:
df.sort_values(by=['Pearson degree assortativity coefficient (PDAC)'])

In [ ]:
df.sort_values(by=['Degree assortativity coefficient(DAC)'])

In [ ]:
for dataset in datasets:
    print(datasets[i].x.shape)